In [117]:
import pandas as pd

In [118]:
def read_csvs():
    df1 = pd.read_csv(r'C:\Users\ville\Code\NHL\CSVs\2013-2014.csv')
    df2 = pd.read_csv(r'C:\Users\ville\Code\NHL\CSVs\2014-2015.csv')
    df3 = pd.read_csv(r'C:\Users\ville\Code\NHL\CSVs\2015-2016.csv')
    df4 = pd.read_csv(r'C:\Users\ville\Code\NHL\CSVs\2016-2017.csv')
    df5 = pd.read_csv(r'C:\Users\ville\Code\NHL\CSVs\2017-2018.csv')
    
    df1['season'] = '2013-2014' # best place to add 'season' attribute without later having to load the dataframes again
    df2['season'] = '2014-2015'
    df3['season'] = '2015-2016'
    df4['season'] = '2016-2017'
    df5['season'] = '2017-2018'

    files = [df1, df2, df3, df4, df5]
    df_all_seasons = pd.concat(files)
    df_all_seasons.reset_index(drop = True, inplace = True)
    return df_all_seasons

#read_csvs()

In [119]:
# OBSOLETE: my API code doesn't have any dates

#df = read_csv() # later remove and add this to get_and_update_df()
#date_string = df.iloc[31,2]
#game_id = 2018021001

"""
def add_season(game):
    df = main_df
    df.reset_index(drop=True, inplace = True) # Probably not needed here
    #print(type(df.game_id)) # type Series
    #print(type(game))       # type Int
    date_string = df.loc[df.game_id == game.values[0], 'start_time'].iloc[0] # iloc for getting a value and not a Series back 
    #date_string = game['start_time']
    date_object = datetime.datetime.strptime(date_string[0:10], '%Y-%m-%d')
    print("add_season")
    print(date_string)
    if date_object.month > 8:
        return str(date_object.year) + "-" + str(date_object.year + 1)
    else:
        return str(date_object.year -1) + "-" + str(date_object.year)
"""
#add_season(2018030413)

'\ndef add_season(game):\n    df = main_df\n    df.reset_index(drop=True, inplace = True) # Probably not needed here\n    #print(type(df.game_id)) # type Series\n    #print(type(game))       # type Int\n    date_string = df.loc[df.game_id == game.values[0], \'start_time\'].iloc[0] # iloc for getting a value and not a Series back \n    #date_string = game[\'start_time\']\n    date_object = datetime.datetime.strptime(date_string[0:10], \'%Y-%m-%d\')\n    print("add_season")\n    print(date_string)\n    if date_object.month > 8:\n        return str(date_object.year) + "-" + str(date_object.year + 1)\n    else:\n        return str(date_object.year -1) + "-" + str(date_object.year)\n'

In [120]:
# OBSOLETE with my API code
"""
def only_regular_time_wins(df):
    df = df[df.final_period == 3] 
    #vector_df = vector_df[vector_df.result != 'x']
    return df
    """

"\ndef only_regular_time_wins(df):\n    df = df[df.final_period == 3] \n    #vector_df = vector_df[vector_df.result != 'x']\n    return df\n    "

In [121]:
# OBSOLETE
def only_regular_season(df):
    df = df[df.game_type == 'R']
    return df

In [122]:
def get_df_and_change_column_names(): # better later than never..
    df = read_csvs()
    df = df.rename({'h_abbr': 'h_team', 'a_abbr': 'a_team'}, axis=1)
    return df
#get_df_and_change_column_names()

In [123]:
def get_new_columns(df):
    return df.columns

In [124]:
def get_games_list(df):
    list = df['game_id'].to_list()
    return list

In [125]:
def sort_df(df):
    df.sort_values(by=['game_id'], inplace = True)
    df.reset_index(drop=True, inplace = True)
    return df

In [126]:
# NOTE: result is after OT, NOT regular time NOR shootout..

def compute_result(game):
    if game['h_goals'] > game['a_goals']:
        return 1
    if game['h_goals'] < game['a_goals']:
        return 0
    else:
        return 'x'  

In [127]:
# USE THIS INSTEAD OF only_regular_time_wins()
def drop_draw(df):
    df.drop(df[df.result == 'x'].index, inplace=True)
    return df

In [128]:
# get running game count for both teams
# https://stackoverflow.com/questions/59693602/python-pandas-count-string-found-in-column-so-far-for-every-row

def running_game_count(df): 
    s = df[['h_team','a_team']].stack()
    df = df.join(s.groupby(s).cumcount().add(1).unstack().add_suffix('_game_no'))
    print('running_game_count')
    return df
#running_game_count(main_df)

In [129]:
# takes season change into account --> can use with CSV's that have multiple seasons of data
# main_df has already called this function so this testing this requires to call get_df_and_change_column_names() or similar

def running_game_count_v2(df):
    #df = get_df_and_change_column_names()
    #print(df.head())
    #df = main_df
    #test_games = list(range(2017030414, 2018020080))
    #df = df[df['game_id'].isin(test_games)]
    df1 = df[df.season == '2013-2014']
    df2 = df[df.season == '2014-2015']    
    df3 = df[df.season == '2015-2016']
    
    df4 = df[df.season == '2016-2017']
    df5 = df[df.season == '2017-2018']
    
    df1 = running_game_count(df1)
    df2 = running_game_count(df2)
    df3 = running_game_count(df3)
    df4 = running_game_count(df4)
    df5 = running_game_count(df5)
    frames = [df1, df2, df3, df4, df5]
    df = pd.concat(frames, ignore_index = True, sort = False) # without sort=False columns will end up in alphabetical order
    print(df.shape)
    return df

#running_game_count_v2(main_df)        

In [130]:
#check the games_list

def get_and_update_df():
    global main_df
    main_df = get_df_and_change_column_names()
    
    main_df = sort_df(main_df)
    main_df['result'] = main_df.apply(lambda game: compute_result(game), axis = 1)
    #main_df['season'] = main_df.apply(lambda game: add_season(game), axis = 1)
    #print('season UPDATED')
    print(main_df.groupby('season').count())
    main_df = running_game_count_v2(main_df)
    
    return main_df
    
# get_and_update_df()

           game_id  h_id  h_team  a_id  a_team  h_goals  a_goals  h_pim  \
season                                                                    
2013-2014     1148  1148    1148  1148    1148     1148     1148   1148   
2014-2015     1230  1230    1230  1230    1230     1230     1230   1230   
2015-2016     1230  1230    1230  1230    1230     1230     1230   1230   
2016-2017     1230  1230    1230  1230    1230     1230     1230   1230   
2017-2018     1271  1271    1271  1271    1271     1271     1271   1271   

           a_pim  h_shots  ...  a_FO_pct  h_blocked  a_blocked  h_tAways  \
season                     ...                                             
2013-2014   1148     1148  ...      1148       1148       1148      1148   
2014-2015   1230     1230  ...      1230       1230       1230      1230   
2015-2016   1230     1230  ...      1230       1230       1230      1230   
2016-2017   1230     1230  ...      1230       1230       1230      1230   
2017-2018   1271  

,game_id,h_id,h_team,a_id,a_team,h_goals,a_goals,h_pim,a_pim,h_shots,...,h_tAways,a_tAways,h_gAways,a_gAways,h_hits,a_hits,season,result,h_team_game_no,a_team_game_no
0,2013020001,8,MTL,10,TOR,3,4,53,47,37,...,7,3,10,5,21,25,2013-2014,0,1,1
1,2013020002,16,CHI,15,WSH,6,4,12,8,35,...,9,1,10,1,30,28,2013-2014,1,1,1
2,2013020003,22,EDM,52,WPG,4,5,15,17,38,...,4,9,9,18,20,24,2013-2014,0,1,1
3,2013020004,4,PHI,10,TOR,1,3,11,21,32,...,9,7,7,12,22,34,2013-2014,0,1,2
4,2013020005,17,DET,7,BUF,2,1,14,6,34,...,7,5,11,9,13,21,2013-2014,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6104,2017021267,53,ARI,24,ANA,0,3,11,15,31,...,4,2,9,1,23,14,2017-2018,0,82,82
6105,2017021268,20,CGY,54,VGK,7,1,6,10,31,...,16,11,14,13,19,16,2017-2018,1,82,82
6106,2017021269,22,EDM,23,VAN,2,2,8,4,33,...,12,13,13,11,12,19,2017-2018,x,82,82
6107,2017021270,26,LAK,25,DAL,2,4,2,6,36,...,2,2,11,6,15,11,2017-2018,0,82,82


In [131]:
# not sure if I need this anymore either
"""
def original_columns():
    df = read_csv()
    columns = list(df.columns)
    return columns
    """

'\ndef original_columns():\n    df = read_csv()\n    columns = list(df.columns)\n    return columns\n    '

In [132]:
#after a road trip averages need some balancing
# some kind of distance/division adjustment at least for h_win_adv (Rangers away at Islanders or at Ducks..)

def get_H_A_averages():
    df = main_df
    #df.drop(['game_id'], 1, inplace = True) # removed obsolete: 'h_id', 'a_id'
    averages = df.mean(axis=0)
    h_goals_adv = averages['h_goals'] - averages['a_goals']
    h_pim_adv   = averages['h_pim'] - averages['a_pim']
    h_shots_adv = averages['h_shots'] - averages['a_shots']
    h_PPG_adv   = averages['h_PPG'] - averages['a_PPG']
    h_PP_oppt_adv=averages['h_PP_oppt'] - averages['a_PP_oppt']
    h_FO_win_pct_adv = averages['h_FO_pct'] - averages['a_FO_pct']
    h_blocked_adv = averages['h_blocked'] - averages['a_blocked']
    h_tAways_adv = averages['h_tAways'] - averages['a_tAways']
    h_gAways_adv = averages['h_gAways'] - averages['a_gAways']
    h_hits_adv = averages['h_hits'] - averages['a_hits']
    h_win_adv = 0.1 # 55/45 in NHL. let's see how to get this without another big mess with updating the df and whatnot

    return h_goals_adv, h_pim_adv, h_shots_adv, h_PPG_adv, h_PP_oppt_adv, h_FO_win_pct_adv, h_blocked_adv,\
                h_tAways_adv, h_gAways_adv, h_hits_adv, h_win_adv
    
#get_H_A_averages()    

In [133]:
def get_avg_columns():
    columns = ['home', 'away', 'gF', 'gA', 'sF', 'sA', 'pimF', 'pimA', 'PPG', 'PP_oppt',\
               'FO_win_pct', 'blocked', 'TA', 'GA', 'hits', 'win_pct']
    return columns

### Home team averages

In [134]:
# filter home games for the current game's home team
def HT_h_games(game_id, no_prev_games):
    h_prev_x_games, a_prev_x_games, h_team, a_team = previous_x_games(game_id, no_prev_games)
    team = h_team
    
    # this filtering might be what slows the program down because its done for every game
    games = h_prev_x_games.loc[h_prev_x_games.h_team.values == team] # notice .values
    
    columns = get_avg_columns()
    df = pd.DataFrame(columns = columns)
    
    h_goals_adv, h_pim_adv, h_shots_adv, h_PPG_adv, h_PP_oppt_adv, h_FO_win_pct_adv, h_blocked_adv,\
    h_tAways_adv, h_gAways_adv, h_hits_adv, h_win_adv\
    = get_H_A_averages()
    
    df['home'] = team
    df['gF'] = games['h_goals'].sum() #let's not get average of averages later
    df['gA'] = games['a_goals'].sum() + h_goals_adv*len(games.h_goals)
    df['pimF'] = games['h_pim'].sum() 
    df['pimA'] = games['a_pim'].sum() + h_pim_adv*len(games.h_goals) 
    df['sF'] = games['h_shots'].sum()
    df['sA'] = games['a_shots'].sum() + h_shots_adv*len(games.h_goals)
    df['PPG'] = games['h_PPG'].sum()
    df['PP_oppt'] = games['h_PP_oppt'].sum()
    df['FO_win_pct'] = games['h_FO_pct'].sum() 
    df['blocked'] = games['h_blocked'].sum()
    df['TA'] = games['h_tAways'].sum()
    df['GA'] = games['h_gAways'].sum()
    df['hits'] = games['h_hits'].sum()
    df['win_pct'] = games.loc[games.result == 1, 'result'].count() # number of home wins (home win 1, away win 0)
    
    return df 
#HT_h_games(2017020074, 1)

In [135]:
# home team away games

def HT_a_games(game_id, no_prev_games):
    h_prev_x_games, a_prev_x_games, h_team, a_team = previous_x_games(game_id, no_prev_games)
    team = h_team
    games = h_prev_x_games.loc[h_prev_x_games.a_team.values == team] # notice .values
    columns = get_avg_columns()
    df = pd.DataFrame(columns = columns)
    h_goals_adv, h_pim_adv, h_shots_adv, h_PPG_adv, h_PP_oppt_adv, h_FO_win_pct_adv, h_blocked_adv,\
    h_tAways_adv, h_gAways_adv, h_hits_adv, h_win_adv\
    = get_H_A_averages()
    
    df['home'] = team
    df['gF'] = games['a_goals'].sum() + h_goals_adv*len(games.a_goals)
    df['gA'] = games['h_goals'].sum() 
    df['pimF'] = games['a_pim'].sum() + h_pim_adv*len(games.a_goals)
    df['pimA'] = games['h_pim'].sum()
    df['sF'] = games['a_shots'].sum() + h_shots_adv*len(games.a_goals)
    df['sA'] = games['h_shots'].sum()
    df['PPG'] = games['a_PPG'].sum() + h_PPG_adv*len(games.a_goals)
    df['PP_oppt'] = games['a_PP_oppt'].sum() + h_PP_oppt_adv*len(games.a_goals)
    df['FO_win_pct'] = games['a_FO_pct'].sum() + h_FO_win_pct_adv*len(games.a_goals)
    df['blocked'] = games['a_blocked'].sum() + h_blocked_adv*len(games.a_goals)
    df['TA'] = games['a_tAways'].sum() + h_tAways_adv*len(games.a_goals)
    df['GA'] = games['a_gAways'].sum() + h_gAways_adv*len(games.a_goals)
    df['hits'] = games['a_hits'].sum() + h_hits_adv*len(games.a_goals)
    df['win_pct'] = games.loc[games.result == 0, 'result'].count() + h_win_adv*len(games.a_goals)
    
    return df
#HT_a_games(2017020074, 5)

In [136]:
# home team's home and away game stats and calculate the averages
# NOTE 19.7: is there a real need for averages or would plain values be enough?

def HT_prev_x_games_avg(game_id, no_prev_games):
    h_df = HT_h_games(game_id, no_prev_games)
    a_df = HT_a_games(game_id, no_prev_games)
    
    columns = get_avg_columns()
    avg_df = pd.DataFrame(columns = columns)
    
    avg_df['home'] = h_df['home']
    avg_df['gF'] = h_df['gF'].values*len(h_df)/ no_prev_games + a_df['gF'].values *len(a_df)/ no_prev_games
    avg_df['gA'] = h_df['gA'].values *len(h_df)/ no_prev_games + a_df['gA'].values *len(a_df)/ no_prev_games
    avg_df['sF'] = h_df['sF'].values *len(h_df)/ no_prev_games + a_df['sF'].values *len(a_df)/ no_prev_games
    avg_df['sA'] = h_df['sA'].values *len(h_df)/ no_prev_games + a_df['sA'].values *len(a_df)/ no_prev_games
    avg_df['pimF'] = h_df['pimF'].values *len(h_df)/ no_prev_games + a_df['pimF'].values *len(a_df)/ no_prev_games
    avg_df['pimA'] = h_df['pimA'].values *len(h_df)/ no_prev_games + a_df['pimA'].values *len(a_df)/ no_prev_games
    avg_df['PPG'] = h_df['PPG'].values *len(h_df)/ no_prev_games + a_df['PPG'].values *len(a_df)/ no_prev_games
    avg_df['PP_oppt'] = h_df['PP_oppt'].values *len(h_df)/ no_prev_games + a_df['PP_oppt'].values *len(a_df)/ no_prev_games
    avg_df['FO_win_pct'] = h_df['FO_win_pct'].values *len(h_df)/ no_prev_games + a_df['FO_win_pct'].values *len(a_df)/ no_prev_games
    avg_df['blocked'] = h_df['blocked'].values *len(h_df)/ no_prev_games + a_df['blocked'].values *len(a_df)/ no_prev_games
    avg_df['TA'] = h_df['TA'].values *len(h_df)/ no_prev_games + a_df['TA'].values *len(a_df)/ no_prev_games
    avg_df['GA'] = h_df['GA'].values *len(h_df)/ no_prev_games + a_df['GA'].values *len(a_df)/ no_prev_games
    avg_df['hits'] = h_df['hits'].values *len(h_df)/ no_prev_games  + a_df['hits'].values *len(a_df)/ no_prev_games
    avg_df['win_pct'] = h_df['win_pct'].values *len(h_df)/ no_prev_games + a_df['win_pct'].values *len(a_df)/ no_prev_games

    return avg_df

#HT_prev_x_games_avg(2017020074, 5)

### Away team averages 


In [137]:
# away team home games

def AT_h_games(game_id, no_prev_games):
    h_prev_x_games, a_prev_x_games, h_team, a_team = previous_x_games(game_id, no_prev_games)
    team = a_team
    games = a_prev_x_games.loc[a_prev_x_games.h_team.values == team] # notice .values
    
    columns = get_avg_columns()
    df = pd.DataFrame(columns = columns)
       
    h_goals_adv, h_pim_adv, h_shots_adv, h_PPG_adv, h_PP_oppt_adv, h_FO_win_pct_adv, h_blocked_adv,\
    h_tAways_adv, h_gAways_adv, h_hits_adv, h_win_adv\
    = get_H_A_averages()
    
    df['away'] = team
    df['gF'] = games['h_goals'].sum() #let's not get average of averages later
    df['gA'] = games['a_goals'].sum() + h_goals_adv*len(games.h_goals)
    df['pimF'] = games['h_pim'].sum() 
    df['pimA'] = games['a_pim'].sum() + h_pim_adv*len(games.h_goals) 
    df['sF'] = games['h_shots'].sum()
    df['sA'] = games['a_shots'].sum() + h_shots_adv*len(games.h_goals)
    df['PPG'] = games['h_PPG'].sum()
    df['PP_oppt'] = games['h_PP_oppt'].sum()
    df['FO_win_pct'] = games['h_FO_pct'].sum() 
    df['blocked'] = games['h_blocked'].sum()
    df['TA'] = games['h_tAways'].sum()
    df['GA'] = games['h_gAways'].sum()
    df['hits'] = games['h_hits'].sum()
    df['win_pct'] = games.loc[games.result == 1, 'result'].count() # number of home wins (home win 1, away win 0)
    return df
#AT_h_games(2017021155, 5)

In [138]:
# away team away games

def AT_a_games(game_id, no_prev_games):
    h_prev_x_games, a_prev_x_games, h_team, a_team = previous_x_games(game_id, no_prev_games)
    team = a_team
    games = a_prev_x_games.loc[a_prev_x_games.a_team.values == team] # notice .values
    columns = get_avg_columns()
    df = pd.DataFrame(columns = columns)
    
    h_goals_adv, h_pim_adv, h_shots_adv, h_PPG_adv, h_PP_oppt_adv, h_FO_win_pct_adv, h_blocked_adv,\
    h_tAways_adv, h_gAways_adv, h_hits_adv, h_win_adv\
    = get_H_A_averages()
    
    df['away'] = team
    df['gF'] = games['a_goals'].sum() + h_goals_adv*len(games.a_goals)
    df['gA'] = games['h_goals'].sum() 
    df['pimF'] = games['a_pim'].sum() + h_pim_adv*len(games.a_goals)
    df['pimA'] = games['h_pim'].sum()
    df['sF'] = games['a_shots'].sum() + h_shots_adv*len(games.a_goals)
    df['sA'] = games['h_shots'].sum()
    df['PPG'] = games['a_PPG'].sum() + h_PPG_adv*len(games.a_goals)
    df['PP_oppt'] = games['a_PP_oppt'].sum() + h_PP_oppt_adv*len(games.a_goals)
    df['FO_win_pct'] = games['a_FO_pct'].sum() + h_FO_win_pct_adv*len(games.a_goals)
    df['blocked'] = games['a_blocked'].sum() + h_blocked_adv*len(games.a_goals)
    df['TA'] = games['a_tAways'].sum() + h_tAways_adv*len(games.a_goals)
    df['GA'] = games['a_gAways'].sum() + h_gAways_adv*len(games.a_goals)
    df['hits'] = games['a_hits'].sum() + h_hits_adv*len(games.a_goals)
    df['win_pct'] = games.loc[games.result == 0, 'result'].count() + h_win_adv*len(games.a_goals)
    
    return df
#AT_a_games(2017020074, 5)

In [139]:
# averages for away team

def AT_prev_x_games_avg(game_id, no_prev_games):
    h_df = AT_h_games(game_id, no_prev_games)
    a_df = AT_a_games(game_id, no_prev_games)
    columns = get_avg_columns()
    avg_df = pd.DataFrame(columns = columns)
    
    avg_df['away'] = a_df['away']
    avg_df['gF'] = h_df['gF'].values*len(h_df)/ no_prev_games + a_df['gF'].values *len(a_df)/ no_prev_games
    avg_df['gA'] = h_df['gA'].values*len(h_df)/ no_prev_games + a_df['gA'].values *len(a_df)/ no_prev_games
    avg_df['pimF'] = h_df['pimF'].values*len(h_df)/ no_prev_games + a_df['pimF'].values *len(a_df)/ no_prev_games
    avg_df['pimA'] = h_df['pimA'].values*len(h_df)/ no_prev_games + a_df['pimA'].values *len(a_df)/ no_prev_games 
    avg_df['sF'] = h_df['sF'].values*len(h_df)/ no_prev_games + a_df['sF'].values *len(a_df)/ no_prev_games
    avg_df['sA'] = h_df['sA'].values*len(h_df)/ no_prev_games + a_df['sA'].values *len(a_df)/ no_prev_games
    avg_df['PPG'] = h_df['PPG'].values*len(h_df)/ no_prev_games + a_df['PPG'].values *len(a_df)/ no_prev_games
    avg_df['PP_oppt'] = h_df['PP_oppt'].values*len(h_df)/ no_prev_games + a_df['PP_oppt'].values *len(a_df)/ no_prev_games
    avg_df['FO_win_pct'] = h_df['FO_win_pct'].values*len(h_df)/ no_prev_games + a_df['FO_win_pct'].values *len(a_df)/ no_prev_games
    avg_df['blocked'] = h_df['blocked'].values*len(h_df)/ no_prev_games + a_df['blocked'].values *len(a_df)/ no_prev_games
    avg_df['TA'] = h_df['TA'].values*len(h_df)/ no_prev_games + a_df['TA'].values *len(a_df)/ no_prev_games
    avg_df['GA'] = h_df['GA'].values*len(h_df)/ no_prev_games + a_df['GA'].values *len(a_df)/ no_prev_games
    avg_df['hits'] = h_df['hits'].values*len(h_df)/ no_prev_games + a_df['hits'].values *len(a_df)/ no_prev_games
    avg_df['win_pct'] = h_df['win_pct'].values*len(h_df)/ no_prev_games + a_df['win_pct'].values *len(a_df)/ no_prev_games

    return avg_df # Don't reset the index while returning it - It'll return a NoneType object - another fun 2h spent :)

#AT_prev_x_games_avg(2017020074, 5)

### Previous x games: V(h-a)

In [140]:
def home_minus_away_avgs(game_id, no_prev_games):
    columns = get_avg_columns()
    df = pd.DataFrame(columns = columns)
    df.insert(loc=0, column='game_id', value = game_id)
    h_df = HT_prev_x_games_avg(game_id, no_prev_games)
    a_df = AT_prev_x_games_avg(game_id, no_prev_games)
    
    df['game_id'] = [game_id] # the square brackets make the int a series. or smth I dunno :)
    df['home'] = h_df['home']
    df['away'] = a_df['away']
    df['gF'] = h_df['gF'] - a_df['gF']
    df['gA'] = h_df['gA'] - a_df['gA']
    df['pimF'] = h_df['pimF'] - a_df['pimF']
    df['pimA'] = h_df['pimA'] - a_df['pimA']    
    df['sF'] = h_df['sF'] - a_df['sF']
    df['sA'] = h_df['sA'] - a_df['sA']
    df['PPG'] = h_df['PPG'] - a_df['PPG']
    df['PP_oppt'] = h_df['PP_oppt'] - a_df['PP_oppt']
    df['FO_win_pct'] = h_df['FO_win_pct'] - a_df['FO_win_pct']
    df['blocked'] = h_df['blocked'] - a_df['blocked']
    df['TA'] = h_df['TA'] - a_df['TA']
    df['GA'] = h_df['GA'] - a_df['GA']
    df['hits'] = h_df['hits'] - a_df['hits']
    df['win_pct'] = h_df['win_pct'] - a_df['win_pct']
    
    return df
#home_minus_away_avgs(2017020074, 5)

## Machine Learning Bitches!!!

In [148]:
def df_to_csv(df):    
    df.to_csv(r'C:\Users\ville\Code\NHL\CSVs\2013-2018-5games.csv')
    print("CSV written")

In [142]:
games = list(range(2018020001, 2018020075))# short test list

#id_list = df['game_id']

def pastamaker(id_list, no_prev_games):
    all_games = []
    #columns = get_avg_columns()

    for game_id in id_list:
        _, h_game_no, _, a_game_no, _ = nth_game_for_teams(game_id)
        if h_game_no > no_prev_games and a_game_no > no_prev_games: 
            game = home_minus_away_avgs(game_id, no_prev_games)
            all_games.append(game)
        else:
            pass
        
    vector_df = pd.concat(all_games)
    
    return vector_df

#pastamaker(games, 2)

In [143]:
# based on game_id returns team names and how many games they have played after the game

def nth_game_for_teams(game_id):
    df = main_df
    h_game_no = (df.loc[df.game_id == game_id]['h_team_game_no'].values) # added .values here as well for Samuli's code
    a_game_no = (df.loc[df.game_id == game_id]['a_team_game_no'].values)
    h_team = (df.loc[df.game_id == game_id]['h_team'].values) # need to compares the value to value (previous_x_games())
    a_team = (df.loc[df.game_id == game_id]['a_team'].values)
    season = (df[df.game_id == game_id]['season'].values)
    return h_team, h_game_no, a_team, a_game_no, season
#nth_game_for_teams(2017020030)

In [144]:
# returns a dataframe with x number of previous games from both teams that played on game == game_id
# need to check if the season also matches in the multi season version

def previous_x_games(game_id, no_prev_games):
    df = main_df
    #print(df.season[0])
    #print(df.iloc[1360:1371,:])
    columns = get_new_columns(df) # new columns, ex: get_original_columns()
    h_prev_x_games = pd.DataFrame(columns = columns)
    a_prev_x_games = pd.DataFrame(columns = columns)
    h_team, h_game_no, a_team, a_game_no, season = nth_game_for_teams(game_id)
    count = 1
    while count < no_prev_games + 1:
        h_prev_game = df.loc[(df.h_team_game_no.values == h_game_no - count) & (df.h_team.values == h_team)
             & (df.season.values == season)]
        if h_prev_game.size <= 0:
            h_prev_game = df.loc[(df.a_team_game_no.values == h_game_no - count) & (df.a_team.values == h_team)
            & (df.season.values == season)]
            
        a_prev_game = df.loc[(df.a_team_game_no.values == a_game_no - count) & (df.a_team.values == a_team)
            & (df.season.values == season)                ]
        if a_prev_game.size <= 0:
            a_prev_game = df.loc[(df.h_team_game_no.values == a_game_no - count) & (df.h_team.values == a_team)
            & (df.season.values == season)]

        h_prev_x_games = h_prev_x_games.append(h_prev_game)
        a_prev_x_games = a_prev_x_games.append(a_prev_game)
        count += 1
    return h_prev_x_games, a_prev_x_games, h_team, a_team

#previous_x_games(2017020033, 2)

In [149]:
# GLOBAL main_df needs to be called before main()
get_and_update_df()

def main(no_prev_games):
        test_games = list(range(2017020001, 2017020090))
        test_games2= [2017021250, 201703111, 2018020001, 2018020015, 2018020018]
        game = [2018020074]
        
        df = main_df
        games = get_games_list(df)

        vector_df = pastamaker(games, no_prev_games)
        vector_df = pd.merge(vector_df, df[['game_id', 'result']], on = 'game_id', how= 'left')
        vector_df = drop_draw(vector_df)
        
        df_to_csv(vector_df)
        return vector_df
    
main(5)

           game_id  h_id  h_team  a_id  a_team  h_goals  a_goals  h_pim  \
season                                                                    
2013-2014     1148  1148    1148  1148    1148     1148     1148   1148   
2014-2015     1230  1230    1230  1230    1230     1230     1230   1230   
2015-2016     1230  1230    1230  1230    1230     1230     1230   1230   
2016-2017     1230  1230    1230  1230    1230     1230     1230   1230   
2017-2018     1271  1271    1271  1271    1271     1271     1271   1271   

           a_pim  h_shots  ...  a_FO_pct  h_blocked  a_blocked  h_tAways  \
season                     ...                                             
2013-2014   1148     1148  ...      1148       1148       1148      1148   
2014-2015   1230     1230  ...      1230       1230       1230      1230   
2015-2016   1230     1230  ...      1230       1230       1230      1230   
2016-2017   1230     1230  ...      1230       1230       1230      1230   
2017-2018   1271  

,game_id,home,away,gF,gA,sF,sA,pimF,pimA,PPG,PP_oppt,FO_win_pct,blocked,TA,GA,hits,win_pct,result
0,2013020076,FLA,LAK,0.452316,1.347684,-1.496186,2.696186,10.469668,16.130332,-0.381241,1.257358,-11.862668,0.071141,2.959568,-2.665412,-14.36294,2.200000e-01,0
1,2013020077,WPG,NJD,0.495367,0.104633,1.792372,7.407628,6.260665,6.539335,0.762482,2.685284,-2.714664,3.057718,0.680864,3.130823,6.12588,3.600000e-01,1
2,2013020080,WSH,EDM,-0.800000,-1.000000,4.600000,1.800000,-6.400000,-5.200000,0.200000,-0.200000,-4.080000,-6.000000,2.200000,-7.400000,-3.00000,0.000000e+00,1
3,2013020081,BUF,MIN,-1.747684,0.547684,-5.096186,15.896186,0.269668,-2.669668,-0.981241,-1.742642,-12.902668,2.671141,-0.840432,-0.265412,7.43706,-3.800000e-01,0
4,2013020082,TOR,MIN,0.800000,0.400000,-1.200000,11.200000,-1.200000,-0.600000,-0.200000,-0.800000,-12.980000,6.200000,2.400000,12.000000,16.20000,1.110223e-16,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5705,2017021266,COL,STL,0.000000,-1.200000,-2.800000,6.000000,0.400000,2.000000,1.200000,0.800000,-12.800000,2.200000,-4.200000,1.800000,-4.40000,0.000000e+00,1
5706,2017021267,ARI,ANA,0.756949,-0.356949,-6.088558,6.488558,2.209003,1.790997,0.056278,0.372074,-4.708005,4.813423,3.878704,-0.996235,4.91118,-1.400000e-01,0
5707,2017021268,CGY,VGK,-2.000000,0.000000,-1.800000,-1.000000,6.800000,-2.400000,-0.600000,-1.000000,2.340000,-3.000000,-3.800000,0.000000,-12.40000,-2.000000e-01,1
5709,2017021270,LAK,DAL,-0.104633,-1.095367,0.792372,-3.392372,0.860665,3.539335,0.762482,1.085284,0.665336,2.457718,-0.919136,2.530823,7.32588,1.600000e-01,0


### Stats explained
* note, '=' is used here in a totally non-programming kinda way. One might even call it the "normal" way.
* note2, for continuous stats i.e. for gF or FO_win_pct the number represents the difference between the averages
between the teams for x amount of games. For example, if in 10 past games MTL (home) scored 50 goals and BUF (away) 35 goals, 
gF would be 1,5 (for MTL)

- home = home team
- away = away team
- gF   = goals scored
- gA   = goals conceded
- sF   = shots made/shot
- sA   = opponent shots
- pimF = penalty minutes taken (exception where negative is better for the home team)
- pimA = penalty minutes drawn (exception where positive is better for the home team
- pp_pctF = 

### ideas for improvement

- use same abbreviations in the CSV (change) and the DF (keep)
- create defs for home and away column data to reuse the code (AT_a_games etc)
- create def to get averages for home and away teams - again to reuse the code
- think how to calculate the FO win percentage
- find out what takeaways and giveaways are